# Installing Libraries

In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importing Libraries

In [2]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout,Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from numpy.random import seed

In [3]:
import nltk
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from collections import Counter
import numpy as np

In [4]:
import preprocess
#import ngrams_sentencepred
import pickle
import model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Data Preparation

In [7]:
filename = '/content/drive/MyDrive/NLP CA-1/human_chat.txt'
file = open(filename, 'rt', errors="ignore")
text=file.readlines()

In [8]:
data=[]

In [9]:
for i in text:
  data.append(preprocess.preprocess(i)[2:])

In [10]:
data1=[i for i in data if i!=[]]

In [11]:
data=data1

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
word_index = tokenizer.word_index
total_unique_words = len(tokenizer.word_index)+1
print(total_unique_words)
print(word_index)

2721
{'i': 1, 'you': 2, 'to': 3, 'the': 4, 'a': 5, 'it': 6, 'of': 7, 's': 8, 'that': 9, 'and': 10, 'what': 11, 'is': 12, 'do': 13, 'in': 14, 'for': 15, 'have': 16, 'like': 17, 'are': 18, 'm': 19, 'how': 20, 'but': 21, 'so': 22, 'good': 23, 'about': 24, 'my': 25, 't': 26, 'hi': 27, 'your': 28, 'be': 29, 'with': 30, 'not': 31, 'was': 32, 'on': 33, 'think': 34, 'going': 35, 'oh': 36, 'haha': 37, 'can': 38, 'just': 39, 'up': 40, 'there': 41, 'we': 42, 'this': 43, 'time': 44, 'at': 45, 'me': 46, 'nice': 47, 'too': 48, 'some': 49, 'work': 50, 'very': 51, 'all': 52, 'really': 53, 'as': 54, 'would': 55, 'been': 56, 'yeah': 57, 'day': 58, 'lot': 59, 'did': 60, 'fun': 61, 'yes': 62, 'any': 63, 'great': 64, 'will': 65, 'they': 66, 'pretty': 67, 'well': 68, 'get': 69, 'out': 70, 'one': 71, 'or': 72, 'more': 73, 'cool': 74, 'if': 75, 'go': 76, 'don': 77, 'from': 78, 'know': 79, 'no': 80, 've': 81, 'much': 82, 'am': 83, 'sounds': 84, 'an': 85, 'love': 86, 'them': 87, 'also': 88, 'try': 89, 'where': 

In [13]:
input_sequences = []
for line in data:
  #print(line)
  token_list = tokenizer.texts_to_sequences([line])[0]
  #print(token_list)
  for i in range(1, len(token_list)): 
    n_gram_seqs = token_list[:i+1]
    input_sequences.append(n_gram_seqs)

In [14]:
print(data)

[['hi'], ['what', 'is', 'your', 'favorite', 'holiday'], ['one', 'where', 'i', 'get', 'to', 'meet', 'lots', 'of', 'different', 'people'], ['what', 'was', 'the', 'most', 'number', 'of', 'people', 'you', 'have', 'ever', 'met', 'during', 'a', 'holiday'], ['hard', 'to', 'keep', 'a', 'count', 'maybe', '25'], ['which', 'holiday', 'was', 'that'], ['i', 'think', 'it', 'was', 'australia'], ['do', 'you', 'still', 'talk', 'to', 'the', 'people', 'you', 'met'], ['not', 'really', 'the', 'interactions', 'are', 'usually', 'short', 'lived', 'but', 'it', 's', 'fascinating', 'to', 'learn', 'where', 'people', 'are', 'coming', 'from', 'and', 'what', 'matters', 'to', 'them'], ['yea', 'me', 'too', 'i', 'feel', 'like', 'god', 'often', 'puts', 'strangers', 'in', 'front', 'of', 'you', 'and', 'gives', 'you', 'an', 'opportunity', 'to', 'connect', 'with', 'them', 'in', 'that', 'moment', 'in', 'deeply', 'meaningful', 'ways', 'do', 'you', 'ever', 'feel', 'like', 'you', 'know', 'things', 'about', 'strangers', 'without

In [15]:
print(len(input_sequences))
print(input_sequences)

18359
[[11, 12], [11, 12, 28], [11, 12, 28, 118], [11, 12, 28, 118, 301], [71, 90], [71, 90, 1], [71, 90, 1, 69], [71, 90, 1, 69, 3], [71, 90, 1, 69, 3, 222], [71, 90, 1, 69, 3, 222, 274], [71, 90, 1, 69, 3, 222, 274, 7], [71, 90, 1, 69, 3, 222, 274, 7, 206], [71, 90, 1, 69, 3, 222, 274, 7, 206, 93], [11, 32], [11, 32, 4], [11, 32, 4, 140], [11, 32, 4, 140, 634], [11, 32, 4, 140, 634, 7], [11, 32, 4, 140, 634, 7, 93], [11, 32, 4, 140, 634, 7, 93, 2], [11, 32, 4, 140, 634, 7, 93, 2, 16], [11, 32, 4, 140, 634, 7, 93, 2, 16, 128], [11, 32, 4, 140, 634, 7, 93, 2, 16, 128, 390], [11, 32, 4, 140, 634, 7, 93, 2, 16, 128, 390, 256], [11, 32, 4, 140, 634, 7, 93, 2, 16, 128, 390, 256, 5], [11, 32, 4, 140, 634, 7, 93, 2, 16, 128, 390, 256, 5, 301], [183, 3], [183, 3, 391], [183, 3, 391, 5], [183, 3, 391, 5, 635], [183, 3, 391, 5, 635, 176], [183, 3, 391, 5, 635, 176, 1299], [104, 301], [104, 301, 32], [104, 301, 32, 9], [1, 34], [1, 34, 6], [1, 34, 6, 32], [1, 34, 6, 32, 392], [13, 2], [13, 2, 20

In [16]:
max_seq_length = max([len(x) for x in input_sequences])
input_seqs = np.array(pad_sequences(input_sequences, maxlen=max_seq_length, padding='pre'))
print(max_seq_length)
print(input_seqs[:5])

154
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0  11  12]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

In [17]:
x_values, labels = input_seqs[:, :-1], input_seqs[:, -1]
y_values = tf.keras.utils.to_categorical(labels, num_classes=total_unique_words)

# Model Building

Model1

In [18]:
model1 = model.create_model1(max_seq_length, total_unique_words)
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 153, 10)           27210     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2721)              274821    
                                                                 
Total params: 346,431
Trainable params: 346,431
Non-trainable params: 0
_________________________________________________________________


Model2

In [19]:
word_model = gensim.models.Word2Vec(data, size=100, min_count = 1, window = 5)

In [20]:
embeddings_matrix = np.zeros((total_unique_words, 100))
for word, i in word_index.items():
   embedding_vector = word_model.wv[word]
   if embedding_vector is not None:
     embeddings_matrix[i] = embedding_vector

In [21]:
model2 = model.create_model2(total_unique_words, embeddings_matrix, max_seq_length)
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 153, 100)          272100    
                                                                 
 bidirectional (Bidirectiona  (None, 153, 512)         731136    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 153, 512)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 512)              1574912   
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense (Dense)               (None, 128)               6

Training the models

In [ ]:
model1.fit(x_values, y_values, epochs=100, verbose=5)

Disadvantges of the model1:

```
The context is not captured since, we used n-grams for training
We can improvise the above model by using word embeddings for representation of words.
```




In [ ]:
model2.fit(x_values, y_values, epochs=100, verbose=5)

Saving the models built

In [ ]:
pickle.dump(model1, open('/content/drive/MyDrive/NLP CA-1/model.pkl', 'wb'))
pickle.dump(model1, open('/content/drive/MyDrive/NLP CA-1/model_word2vec.pkl', 'wb'))

# Predictions

In [ ]:
model_1 = pickle.load(open('/content/drive/MyDrive/NLP CA-1/model.pkl', 'rb'))
model_2 = pickle.load(open('/content/drive/MyDrive/NLP CA-1/model_word2vec.pkl', 'rb'))

In [23]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0))
        output_word = ""
        for word,index in tokenizer.word_index.items():
            #print(word, index, predicted)
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

Predictions using model1

In [24]:
print (generate_text("what are you", 3, model_1, max_seq_length))

What Are You Up To Eat


In [42]:
print (generate_text("hi I", 4, model_1, max_seq_length))

Hi I Don T Lose Any


In [30]:
print (generate_text("How is", 5, model_1, max_seq_length))

How Is Your Day Going To Cook


Predictions using model2

In [41]:
print (generate_text("Hi I", 3, model_2, max_seq_length))

Hi I M Doing Well


In [44]:
print (generate_text('what is your', 2, model_2, max_seq_length))

What Is Your Favorite Place


In [36]:
print (generate_text("How are",1 , model_2, max_seq_length))

How Are You
